In [15]:
from sklearn.utils.class_weight import compute_sample_weight
#from torch.utils.tensorboard import SummaryWriter
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
#import lightgbm as lgbm
from sklearn import metrics
import torch.nn as nn
from torch import Tensor
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import pandas as pd
# Importing the utils.py file from the class_files app.
from utils import *
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report
import torchtuples as tt
import warnings
warnings.filterwarnings("ignore")

In [65]:
from Type_simulation import *

In [79]:
class Loss(nn.Module):

    def __init__(self,  alpha, beta=1):
        super().__init__()
        self.beta = beta
        self.alpha = alpha
        self.loss_classif = nn.BCELoss()
        self.loss_wass = WassLoss()  # IPM

    def forward(self, phi_t, sigma, y_train):
        y_train = y_train.reshape(-1, 1)
        sigma = sigma.reshape(-1, 1)
        loss_classif = self.loss_classif(sigma, y_train)
        loss_wass = self.loss_wass(phi_t)  # Wasserstein Loss
        self.wd = loss_wass.item()
        self.cl = loss_classif.item()
        return self.beta*loss_classif + self.alpha * loss_wass  #


# It creates a class that inherits from nn.Module.
class WassLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, psi: Tensor) -> Tensor:
        a, b = sepr_repr(psi)
        self.psi0 = a
        self.psi1 = b
        return SinkhornDistance(eps=0.001, max_iter=100, reduction=None)(a, b)

In [80]:
class NetClassif(nn.Module):
    def __init__(self, in_features, encoded_features):
        super().__init__()
        self.phi = nn.Sequential(
            nn.Linear(in_features-1, 32),  nn.LeakyReLU(),
            nn.Linear(32, 32),  nn.ReLU(),
            nn.Linear(32, 28),  nn.LeakyReLU(),
            nn.Linear(28, encoded_features),
        )
        self.psi = nn.Sequential(
            nn.Linear(encoded_features + 1, 128), nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 50),  nn.ReLU(),
            nn.Linear(50, 1),
        )
        self.loss_classif = nn.BCELoss()
        self.loss_wass = WassLoss()  # IPM

    def forward(self, input):
        x, t = get_data(input)
        self.input = input
        t = t.reshape(-1, 1)
        phi = self.phi(x)
        phi_t = torch.cat((phi, t), 1)
        sigma = nn.Sigmoid()(self.psi(phi_t))
        return phi_t, sigma

In [81]:
class ClassifBase(tt.Model):
    """Base class for classification models.
    Essentially same as torchtuples.Model,
    """

    def __init__(self, net, loss=None, optimizer=None, device=None):
        super().__init__(net, loss, optimizer, device)

    def predict_proba(self, input,  **kwargs):
        x, t = get_data(input)
        self.input = input
        t = t.reshape(-1, 1)
        phi = self.net.phi(x)
        phi_t = torch.cat((phi, t), 1)
        sigma = nn.Sigmoid()(self.net.psi(phi_t))
        return sigma.detach().cpu().numpy()

In [82]:
class DataLoader():

    def __init__(self):
        super().__init__()
        self.path = "dataclassif"  # params_sim['path_data']

    def load_data_sim_benchmark(self):

        df = pd.read_csv(self.path + ".csv")
        df = reduce_mem_usage(df)
        self.df = df
        dim = df.shape[1]-9

        x_z_list = ["X" + str(i) for i in range(1, dim + 1)] + ["tt"]
        leave = x_z_list + ["Y_f", "Y_cf", "Y_0", "Y_1", "pi_0", "pi_1"]

        ##
        rs = ShuffleSplit(test_size=.4, random_state=0)
        df_ = df[leave].copy()

        for train_index, test_index in rs.split(df_):
            df_train = df_.drop(test_index)
            df_test = df_.drop(train_index)
            df_val = df_test.sample(frac=0.2)
            df_test = df_test.drop(df_val.index)

        counter_list = ["Y_cf", "Y_0", "Y_1", "pi_0", "pi_1"]
        y_train_classif = df_train['Y_f'].values.astype("float32")
        y_val_classif = df_val['Y_f'].values.astype("float32")
        y_test_classif = df_test['Y_f'].values.astype("float32")

        counter_train = df_train[counter_list].values.astype("float32")
        counter_val = df_val[counter_list].values.astype("float32")
        counter_test = df_test[counter_list].values.astype("float32")

        train = (df_train[x_z_list].values.astype("float32"), y_train_classif)
        val = (
            df_val[x_z_list].values.astype("float32"),
            y_val_classif,
        )

        x_test = df_test[x_z_list].values.astype("float32")

        # SPlit data for OURS
        self.x_train, self.y_train, self.train, self.val,\
            self.y_test, self.x_test, self.counter_train, self.counter_test, self.counter_val = \
            train[0], train[1], train, val, y_test_classif, x_test, counter_train, counter_test, counter_val

        self.x_train = torch.from_numpy(self.x_train).float()
        self.y_train = torch.from_numpy(self.y_train).float().view(-1, 1)
        self.x_val = torch.from_numpy(self.val[0]).float()
        self.y_val = torch.from_numpy(self.val[1]).float().view(-1, 1)
        self.x_test = torch.from_numpy(self.x_test).float()
        self.y_test = torch.from_numpy(self.y_test).float().view(-1, 1)

        #  SPlit data for benchmarking

        def get_separ_data(x):
            mask_1 = x["tt"] == 1
            mask_0 = x["tt"] == 0
            x_1 = x[mask_1].drop(columns="tt")
            x_0 = x[mask_0].drop(columns="tt")
            return x_0, x_1

        df_train_0,  df_train_1 = get_separ_data(df_train)
        df_val_0, df_val_1 = get_separ_data(df_val)
        x_z_list = ["X" + str(i) for i in range(1, dim + 1)]
        self.x_train_0 = torch.from_numpy(
            df_train_0[x_z_list].values.astype("float32")).float()
        self.x_train_1 = torch.from_numpy(
            df_train_1[x_z_list].values.astype("float32")).float()
        self.x_val_0 = torch.from_numpy(
            df_val_0[x_z_list].values.astype("float32")).float()
        self.x_val_1 = torch.from_numpy(
            df_val_1[x_z_list].values.astype("float32")).float()

        self.y_train_0 = torch.from_numpy(
            df_train_0["Y_f"].values.astype("float32")).float().view(-1, 1)
        self.y_train_1 = torch.from_numpy(
            df_train_1["Y_f"].values.astype("float32")).float().view(-1, 1)
        self.y_val_0 = torch.from_numpy(
            df_val_0["Y_f"].values.astype("float32")).float().view(-1, 1)
        self.y_val_1 = torch.from_numpy(
            df_val_1["Y_f"].values.astype("float32")).float().view(-1, 1)

        self.counter_train_0 = torch.from_numpy(
            df_train_0[counter_list].values.astype("float32")).float()
        self.counter_train_1 = torch.from_numpy(
            df_train_1[counter_list].values.astype("float32")).float()
        self.counter_val_0 = torch.from_numpy(
            df_val_0[counter_list].values.astype("float32")).float()
        self.counter_val_1 = torch.from_numpy(
            df_val_1[counter_list].values.astype("float32")).float()

    def get_data(self):
        self.load_data_sim_benchmark()
        return self

In [83]:
class ClassifCaus(nn.Module):

    def __init__(self, params_classifcaus):
        super().__init__()
        encoded_features = params_classifcaus['encoded_features']
        alpha_wass = params_classifcaus['alpha_wass']
        batch_size = params_classifcaus['batch_size']
        epochs = params_classifcaus['epochs']
        lr = params_classifcaus['lr']
        patience = params_classifcaus['patience']

        self.data = DataLoader().get_data()

        self.in_features = self.data.x_train.shape[1]
        self.encoded_features = encoded_features
        self.net = NetClassif(self.in_features, self.encoded_features)

        self.alpha_wass = alpha_wass
        self.lr = lr
        self.loss = Loss(self.alpha_wass, 1)
        self.metrics = dict(loss_classif=Loss(0, 1), loss_wass=Loss(1, 0))
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = ClassifBase(
            net=self.net, loss=self.loss, optimizer=torch.optim.Adam, device=None)

        self.patence = patience
        self.callbacks = [tt.cb.EarlyStopping(patience=patience)]
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=self.lr)

    def fit_model(self):
        self.model.fit(input=self.data.x_train, target=self.data.y_train,
                       val_data=(self.data.x_val, self.data.y_val),
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       callbacks=self.callbacks,
                       metrics=self.metrics)
        return self

    def pred_t(self, x_test, t, y_test, p_test):
        # drop x_test[:,-1] and replace it by t * 1.
        tt = torch.ones(x_test.shape[0], 1) * t
        x_test_t = np.concatenate((x_test[:, :-1], tt.reshape(-1, 1)), axis=1)
        x_test_t = torch.from_numpy(x_test_t).float()
        y_test = y_test.squeeze().numpy()
        p_pred_t = self.model.predict_proba(x_test_t).squeeze()
        y_pred_t = (p_pred_t > 0.5) * 1.0
        acc = metrics.accuracy_score(y_test, y_pred_t)
        cf_m = metrics.confusion_matrix(y_test, y_pred_t)
        f1_s = metrics.f1_score(y_test, y_pred_t)
        auc = metrics.roc_auc_score(y_test, p_pred_t)
        kl = KL(p_test, p_pred_t)
        std_diff =std_diff_metric(p_test, p_pred_t)
        report = classification_report(y_test, y_pred_t)
        return acc, cf_m, f1_s, auc, kl, p_pred_t, report, std_diff

In [84]:
params_classifcaus = {
    "encoded_features": 25,
    "alpha_wass": 0.01,
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.001,
    "patience": 7,
}

In [85]:
param_sim = {
    'n_features': 25,
    'n_classes': 2,
    'n_samples': 1000,
    'beta': [0.1, 0.1, 0.3],
    'coef_tt': 1.1,
    'rho': 15
}
idx = np.arange(param_sim['n_features'])
param_sim['beta'] = (-1) ** idx * np.exp(-idx / 20.)
param_sim['alpha'] = (-1) ** idx * np.exp(-idx / 20.) /3

In [68]:
simu = Simulation_mod(param_sim)
simu.simule(wd_para = 0)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,tt,Y_f,Y_cf,Y_0,Y_1,pi_0,pi_1,pi_f,pi_cf,id
0,0.840774,-2.953608,-1.894213,2.387962,4.276957,1.813672,0.734181,-0.534431,-1.120795,-0.662566,...,0,0,0,0,0,0.073326,0.192060,0.073326,0.192060,0
1,1.164448,2.710316,-0.298409,0.098235,5.597416,1.106360,2.991877,-2.866985,3.994747,-1.915142,...,1,1,1,1,1,0.968901,0.989429,0.989429,0.968901,1
2,1.311295,-2.982420,-3.726019,-2.025751,2.266880,2.895718,-0.334845,-6.615137,-3.001232,5.555681,...,1,1,1,1,1,0.817807,0.930962,0.930962,0.817807,2
3,0.303825,-6.269872,-2.014411,-5.835530,-3.326133,-8.582165,-3.893511,-3.805137,-5.461583,-1.689110,...,0,1,1,1,1,0.999525,0.999842,0.999525,0.999842,3
4,3.890676,3.721534,0.714977,0.218185,-2.493685,-0.372298,-0.862002,1.951581,0.884283,1.662861,...,1,0,0,0,0,0.184167,0.404110,0.404110,0.184167,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6.626844,-2.259308,3.129076,3.421506,4.722830,2.061740,3.738173,4.188156,9.450981,3.691438,...,1,1,1,1,1,0.999997,0.999999,0.999999,0.999997,995
996,-4.317431,-1.281455,4.359847,1.529762,-3.497465,3.134878,2.234285,3.315011,6.942328,3.291208,...,1,1,1,1,1,0.980267,0.993344,0.993344,0.980267,996
997,7.147107,-0.822083,-4.650697,-0.139623,1.085023,1.027633,0.393668,-0.454934,-1.117605,0.341220,...,1,1,1,1,1,0.998410,0.999470,0.999470,0.998410,997
998,1.151042,-1.942032,4.027250,2.555851,-2.475364,-0.590037,-2.048190,0.444736,-0.194229,-2.258305,...,0,1,1,1,1,0.962206,0.987094,0.962206,0.987094,998


In [86]:
classcaus = ClassifCaus (params_classifcaus= params_classifcaus)

Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%


In [88]:
data = DataLoader().get_data()

Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%


In [90]:
data.x_train.shape

torch.Size([600, 26])

In [46]:
classcaus.data.x_val_1.shape

torch.Size([38, 25])

In [98]:
from Type_simulation import *
from Benchmark import *

ModuleNotFoundError: No module named 'Benchmark'

In [99]:
params_classifcaus = {
    "encoded_features": 25,
    "alpha_wass": 0.01,
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.001,
    "patience": 7,
}

In [100]:
param_sim = {
    'n_features': 25,
    'n_classes': 2,
    'n_samples': 1000,
    'beta': [0.1, 0.1, 0.3],
    'coef_tt': 1.1,
    'rho': 15
}
idx = np.arange(param_sim['n_features'])
param_sim['beta'] = (-1) ** idx * np.exp(-idx / 20.)
param_sim['alpha'] = (-1) ** idx * np.exp(-idx / 20.) /3

In [96]:
fit_param_classcaus = {
    "batch_size" : params_classifcaus["batch_size"],
    "epochs" : params_classifcaus["epochs"],
    "callbacks" : [tt.cb.EarlyStopping(patience=params_classifcaus["patience"])],
    "metrics" : dict(loss_classif=Loss(0, 1), loss_wass=Loss(1, 0))
}


In [101]:
simu = Simulation_mod(param_sim)
data = simu.simule(wd_para=0)

In [115]:
X_shift_tt = np.concatenate((simu.X_shift, simu.tt.reshape(simu.n_samples,1)), axis = 1)
outcome_f = np.asarray(data["Y_f"])

In [97]:
classcaus = ClassifCaus(params_classifcaus)

Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%


In [118]:
classcaus.fit_model()

0:	[0s / 0s],		train_loss: 0.6920,	train_loss_classif: 0.6912,	train_loss_wass: 0.0837,	val_loss: 0.6890,	val_loss_classif: 0.6883,	val_loss_wass: 0.0661
1:	[0s / 0s],		train_loss: 0.6878,	train_loss_classif: 0.6874,	train_loss_wass: 0.0473,	val_loss: 0.6851,	val_loss_classif: 0.6844,	val_loss_wass: 0.0727
2:	[0s / 0s],		train_loss: 0.6828,	train_loss_classif: 0.6822,	train_loss_wass: 0.0610,	val_loss: 0.6793,	val_loss_classif: 0.6782,	val_loss_wass: 0.1106
3:	[0s / 0s],		train_loss: 0.6758,	train_loss_classif: 0.6745,	train_loss_wass: 0.1249,	val_loss: 0.6715,	val_loss_classif: 0.6700,	val_loss_wass: 0.1466
4:	[0s / 0s],		train_loss: 0.6681,	train_loss_classif: 0.6660,	train_loss_wass: 0.2138,	val_loss: 0.6616,	val_loss_classif: 0.6589,	val_loss_wass: 0.2675
5:	[0s / 0s],		train_loss: 0.6557,	train_loss_classif: 0.6534,	train_loss_wass: 0.2264,	val_loss: 0.6550,	val_loss_classif: 0.6523,	val_loss_wass: 0.2695
6:	[0s / 0s],		train_loss: 0.6501,	train_loss_classif: 0.6483,	train_loss_wa

ClassifCaus(
  (net): NetClassif(
    (phi): Sequential(
      (0): Linear(in_features=25, out_features=32, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
      (4): Linear(in_features=32, out_features=28, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Linear(in_features=28, out_features=25, bias=True)
    )
    (psi): Sequential(
      (0): Linear(in_features=26, out_features=128, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=128, out_features=50, bias=True)
      (5): ReLU()
      (6): Linear(in_features=50, out_features=1, bias=True)
    )
    (loss_classif): BCELoss()
    (loss_wass): WassLoss()
  )
  (loss): Loss(
    (loss_classif): BCELoss()
    (loss_wass): WassLoss()
  )
)

In [1]:
from Type_simulation import *
from Benchmark import *

In [8]:
param_sim = {
    'n_features': 25,
    'n_classes': 2,
    'n_samples': 1000,
    'beta': [0.1, 0.1, 0.3],
    'coef_tt': 1,
    'rho': 15
}
idx = np.arange(param_sim['n_features'])
param_sim['beta'] = (-1) ** idx * np.exp(-idx / 20.)
param_sim['alpha'] = (-1) ** idx * np.exp(-idx / 20.) /3

params_classifcaus = {
    "encoded_features": 25,
    "alpha_wass": 0.01,
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.001,
    "patience": 7,
}


In [9]:
simu = Simulation_mod(param_sim)
simu.simule(wd_para = 0, function_type="linear", coef_tt_const=False)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,tt,Y_f,Y_cf,Y_0,Y_1,pi_0,pi_1,pi_f,pi_cf,id
0,4.582232,2.946846,0.328854,3.043115,3.018191,5.048214,-3.831565,-3.887028,1.430880,0.681834,...,1,1,0,0,1,0.014447,0.932338,0.932338,0.014447,0
1,1.738480,-2.836754,-2.582598,2.446942,1.907152,1.605542,-3.202857,-0.430623,8.329603,5.882438,...,1,1,1,1,1,0.737756,0.999622,0.999622,0.737756,1
2,-0.580838,-0.117542,1.340701,0.859052,1.722677,-3.081864,-7.941982,-5.880817,-0.658373,-2.750866,...,0,1,1,1,1,0.809718,0.999750,0.809718,0.999750,2
3,-0.190883,1.875602,4.973408,8.703703,8.092241,0.449731,1.449753,8.063242,3.578876,-0.417627,...,1,1,1,1,1,0.994192,0.999994,0.999994,0.994192,3
4,1.586204,-2.075733,10.249707,11.834051,2.600053,-1.764994,-7.184622,-5.379288,1.374429,9.632008,...,1,0,0,0,0,0.000008,0.007826,0.007826,0.000008,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.815832,-0.712875,-2.256611,-6.475482,-0.497559,-2.351044,2.772049,3.848931,0.864651,6.771720,...,1,1,1,1,1,0.988691,0.999988,0.999988,0.988691,995
996,-1.942261,-4.051602,-3.765366,5.065416,6.929367,3.157491,8.792586,0.854118,1.117248,-1.238569,...,1,1,1,1,1,0.999975,1.000000,1.000000,0.999975,996
997,1.461450,-0.121243,3.242743,0.685303,10.894464,-2.411443,-4.502560,-7.154692,-2.923953,-0.976134,...,0,1,1,1,1,1.000000,1.000000,1.000000,1.000000,997
998,2.157961,9.763945,6.510948,0.769337,-1.108967,-2.415462,-2.650337,7.813920,2.307095,1.967788,...,1,1,0,0,1,0.011987,0.919382,0.919382,0.011987,998


In [10]:
classcaus = Classcaus(simu, params_classifcaus)

In [11]:
bench_one = Benchmark(simu)

In [12]:
bench_one.prep_bench(test_size=0.25)

In [45]:
bench_one.predict_one_model(classcaus, type_model="classcaus" )

torch.Size([750, 26])
torch.Size([750, 1])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
0:	[0s / 0s],		train_loss: 0.7000,	train_loss_classif: 0.6990,	train_loss_wass: 0.0990
1:	[0s / 0s],		train_loss: 0.6934,	train_loss_classif: 0.6925,	train_loss_wass: 0.0817
2:	[0s / 0s],		train_loss: 0.6881,	train_loss_classif: 0.6875,	train_loss_wass: 0.0667
3:	[0s / 0s],		train_loss: 0.6819,	train_loss_classif: 0.6813,	train_loss_wass: 0.0616
4:	[0s / 0s],		train_loss: 0.6712,	train_loss_classif: 0.6705,	train_loss_wass: 0.0670
5:	[0s / 0s],		train_loss: 0.6472,	train_loss_classif: 0.6464,	train_loss_wass: 0.0803
6:	[0s / 0s],		train_loss: 0.6038,	train_loss_classif: 0.6028,	train_loss_wass: 0.0984


In [47]:
bench_one.predict_proba_p1

array([0.4600849 , 0.46022022, 0.45991755, 0.4599883 , 0.46011376,
       0.45996228, 0.4603876 , 0.4600655 , 0.46084446, 0.46191   ,
       0.46093684, 0.46071175, 0.46072772, 0.46052256, 0.46079496,
       0.4598729 , 0.46067747, 0.46025932, 0.45996922, 0.46059334,
       0.46121794, 0.46150365, 0.46144304, 0.46089572, 0.4604994 ,
       0.46177396, 0.46282536, 0.46083337, 0.46164304, 0.46053782,
       0.46027482, 0.45980552, 0.46095437, 0.4619468 , 0.46111867,
       0.46031117, 0.46085444, 0.4607757 , 0.46046007, 0.46090555,
       0.46080658, 0.46008176, 0.45998436, 0.46091932, 0.4611272 ,
       0.46116617, 0.46220568, 0.4614463 , 0.46115994, 0.46159935,
       0.46168736, 0.46232414, 0.4608562 , 0.4617321 , 0.46194124,
       0.46200147, 0.46153057, 0.46095356, 0.46094388, 0.46041074,
       0.46054342, 0.46022317, 0.46075055, 0.46050754, 0.46048346,
       0.46020573, 0.45987582, 0.46235752, 0.46104944, 0.46012744,
       0.4609094 , 0.4601944 , 0.46096137, 0.4614932 , 0.46010

In [13]:
bench_one.benchmark_one_model(model=classcaus,type_model="classcaus", name_model="classcaus")

torch.Size([750, 26])
torch.Size([750, 1])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
0:	[0s / 0s],		train_loss: 0.7040,	train_loss_classif: 0.7023,	train_loss_wass: 0.1715
1:	[0s / 0s],		train_loss: 0.6806,	train_loss_classif: 0.6791,	train_loss_wass: 0.1579
2:	[0s / 0s],		train_loss: 0.6472,	train_loss_classif: 0.6456,	train_loss_wass: 0.1645
3:	[0s / 0s],		train_loss: 0.6238,	train_loss_classif: 0.6219,	train_loss_wass: 0.1909
4:	[0s / 0s],		train_loss: 0.6016,	train_loss_classif: 0.5995,	train_loss_wass: 0.2099
5:	[0s / 0s],		train_loss: 0.5791,	train_loss_classif: 0.5770,	train_loss_wass: 0.2142
6:	[0s / 0s],		train_loss: 0.5512,	train_loss_classif: 0.5488,	train_loss_wass: 0.2477


,accu_0,accu_1,MSE_p0,MSE_p1,PEHE,accu_sign_cate
classcaus,0.508,0.172,0.186359,0.235927,0.507797,0.016


In [16]:
bench_one.cate_predict

array([-1.9315481e-03, -5.5617094e-04, -7.5587630e-04, -1.4562309e-03,
       -2.4817884e-03, -1.3597310e-03, -1.0780990e-03, -3.3727288e-04,
       -1.2394786e-03, -1.3014078e-03, -1.1755228e-03, -1.4871061e-03,
       -5.4961443e-04, -1.0651946e-03, -7.2869658e-04, -2.1170974e-03,
       -1.5206039e-03, -6.3607097e-04, -2.9477477e-04, -9.7396970e-04,
       -1.4373660e-03, -7.1850419e-04, -1.2194514e-03, -3.1650066e-05,
       -1.2290776e-03, -1.6688406e-03, -2.0781755e-03, -7.5212121e-04,
       -7.5927377e-04, -8.1089139e-04, -5.7670474e-04, -7.8323483e-04,
       -6.7359209e-04, -9.4524026e-04, -8.4528327e-04, -2.3219585e-03,
       -1.6062558e-03, -9.8836422e-04, -6.2751770e-04, -7.1108341e-04,
       -1.5888512e-03, -1.0793209e-03, -1.2972653e-03, -4.4479966e-04,
       -1.6540289e-03, -1.1005998e-04, -7.4657798e-04, -1.1738539e-03,
       -1.3340116e-03, -2.0594597e-03, -1.0328293e-03, -9.3141198e-04,
       -1.3835430e-03, -1.8362403e-03, -9.5438957e-04, -4.6092272e-04,
      

In [10]:
X = torch.from_numpy(X).float()

In [9]:
tt = simu.tt

In [13]:
tt.reshape(-1, 1).shape

(1000, 1)